In [1]:
pip install pyamg

  Using cached pyamg-4.0.0.tar.gz (717 kB)
  Running setup.py clean for pyamg
Failed to build pyamg
    Running setup.py install for pyamg: started
    Running setup.py install for pyamg: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\ASUS\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Public\\Documents\\Wondershare\\CreatorTemp\\pip-install-r16v3_9j\\pyamg\\setup.py'"'"'; __file__='"'"'C:\\Users\\Public\\Documents\\Wondershare\\CreatorTemp\\pip-install-r16v3_9j\\pyamg\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Public\Documents\Wondershare\CreatorTemp\pip-wheel-daw1sq6d'
       cwd: C:\Users\Public\Documents\Wondershare\CreatorTemp\pip-install-r16v3_9j\pyamg\
  Complete output (132 lines):
  12cf36e46f091253710f5fc5d3868b3ca78d515c
  4.0.0
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\pyamg
  copying pyamg\blackbox.py -> build\lib.win-amd64-3.7\pyamg

# Solving a Fourth Order Elliptic Singular Perturbation Problem

$$
\left\{\begin{array}{lr}
\varepsilon^{2} \Delta^{2} u-\Delta u=f & \text { in } \Omega \\
u=\partial_{n} u=0 & \text { on } \partial \Omega
\end{array}\right.
$$

In [12]:
from skfem import *
import numpy as np
from skfem.visuals.matplotlib import draw, plot
from skfem.utils import solver_iter_krylov
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem import *
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from utils import *
plt.rcParams['figure.dpi'] = 200

pi = np.pi
sin = np.sin
cos = np.cos
exp = np.exp

## Forms and errors

In [13]:
@BilinearForm
def a_load(u, v, w):
    '''
    for $a_{h}$
    '''
    return ddot(dd(u), dd(v))


@BilinearForm
def b_load(u, v, w):
    '''
    for $b_{h}$
    '''
    return dot(grad(u), grad(v))


@BilinearForm
def wv_load(u, v, w):
    '''
    for $(\nabla \chi_{h}, \nabla_{h} v_{h})$
    '''
    return dot(grad(u), grad(v))


@BilinearForm
def laplace(u, v, w):
    '''
    for $(\nabla w_{h}, \nabla \chi_{h})$
    '''
    return dot(grad(u), grad(v))

# Numerical results
> setting boundary condition: $u=0 \ on \ \partial \Omega$

## Parameters 
$$
\tilde{a}_{h}\left(u_{h}, v_{h}\right):=\left(\nabla_{h}^{2} u_{h}, \nabla_{h}^{2} v_{h}\right)-\sum_{F \in \mathcal{F}_{h}^{\partial}}\left(\partial_{n n}^{2} u_{h}, \partial_{n} v_{h}\right)_{F}-\sum_{F \in \mathcal{F}_{h}^{\partial}}\left(\partial_{n} u_{h}, \partial_{n n}^{2} v_{h}\right)_{F}+\sum_{F \in \mathcal{F}_{h}^{\partial}} \frac{\sigma}{h_{F}}\left(\partial_{n} u_{h}, \partial_{n} v_{h}\right)_{F}
$$


- `sigma` in $\sum_{F \in \mathcal{F}_{h}^{\partial}} \frac{\sigma}{h_{F}}\left(\partial_{n} u_{h}, \partial_{n} v_{h}\right)_{F}$

## Example 1
$$
u\left(x_{1}, x_{2}\right)=\left(\sin \left(\pi x_{1}\right) \sin \left(\pi x_{2}\right)\right)^{2}
$$

In [16]:
@LinearForm
def f_load(v, w):
    '''
    for $(f, x_{h})$
    '''
    pix = pi * w.x[0]
    piy = pi * w.x[1]
    lu = 2 * (pi)**2 * (cos(2 * pix) * ((sin(piy))**2) + cos(2 * piy) *
                        ((sin(pix))**2))
    llu = -8 * (pi)**4 * (cos(2 * pix) * sin(piy)**2 + cos(2 * piy) *
                          sin(pix)**2 - cos(2 * pix) * cos(2 * piy))
    return (epsilon**2 * llu - lu) * v


def exact_u(x, y):
    return (sin(pi * x) * sin(pi * y))**2


def dexact_u(x, y):
    dux = 2 * pi * cos(pi * x) * sin(pi * x) * sin(pi * y)**2
    duy = 2 * pi * cos(pi * y) * sin(pi * x)**2 * sin(pi * y)
    return dux, duy


def ddexact(x, y):
    duxx = 2 * pi**2 * cos(pi * x)**2 * sin(pi * y)**2 - 2 * pi**2 * sin(
        pi * x)**2 * sin(pi * y)**2
    duxy = 2 * pi * cos(pi * x) * sin(pi * x) * 2 * pi * cos(pi * y) * sin(
        pi * y)
    duyx = duxy
    duyy = 2 * pi**2 * cos(pi * y)**2 * sin(pi * x)**2 - 2 * pi**2 * sin(
        pi * y)**2 * sin(pi * x)**2
    return duxx, duxy, duyx, duyy

### P1 element

#### Without penalty (Problem1)

In [17]:
refine_time = 6
epsilon_range = 5
element_type = 'P1'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type, f_load)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))

element_type: P1


TypeError: solve_problem1() takes from 1 to 2 positional arguments but 3 were given

#### With penalty (Problem2)

In [7]:
sigma = 5
element_type = 'P1'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))


epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.72  0.82  0.73  0.72
2^-3  2.25  1.80  1.07  1.02
2^-4  2.24  1.95  1.04  1.01
2^-5  2.11  1.99  1.02  1.00
2^-6  2.04  2.00  1.01  1.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.28  0.55  0.23  0.47
2^-3  2.31  1.56  0.69  1.48
2^-4  2.24  1.90  0.98  1.79
2^-5  1.85  2.18  1.36  1.95
2^-6  1.83  2.20  1.46  1.80
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  1.27  0.54  0.22  0.46
2^-3  2.30  1.50  0.60  1.45
2^-4  2.37  1.67  0.66  1.65
2^-5  2.23  1.66  0.62  1.65
2^-6  2.11  1.61  0.58  1.60
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.27  0.54  0.22  0.46
2^-3  2.30  1.50  0.60  1.45
2^-4  2.37  1.67  0.66  1.65
2^-5  2.23  1.66  0.62  1.65
2^-6  2.11  1.61  0.58  1.60
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  1.27  0.54  0.22  0.46
2^-3  2.30  1.50  0.60  1.45
2^-4  2.37  1.67  0.66  1.65
2^-5  2.23  1.66  0.62  1.65
2^-6  2.11  1.61  0.58  1.60


### P2 element

#### Without penalty (Problem1)

In [8]:
refine_time = 6
epsilon_range = 5
element_type = 'P2'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))
        
uh0_no_penalty = uh0

element_type: P2
epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.82  1.71  1.03  0.98
2^-3  1.73  1.74  0.94  0.90
2^-4  1.90  1.88  0.99  0.97
2^-5  1.97  1.96  1.00  0.99
2^-6  1.99  1.99  1.00  1.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  2.83  1.88  0.93  1.75
2^-3  2.62  1.99  0.91  1.81
2^-4  1.11  2.04  1.08  1.71
2^-5  0.92  1.72  1.05  1.30
2^-6  1.47  1.60  0.90  1.00
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  2.86  1.87  0.91  1.77
2^-3  3.06  1.92  0.85  1.87
2^-4  3.36  1.94  0.93  1.91
2^-5  3.28  1.98  0.98  1.97
2^-6  3.12  1.99  0.99  1.99
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  2.86  1.87  0.91  1.77
2^-3  3.06  1.92  0.85  1.87
2^-4  3.37  1.94  0.93  1.91
2^-5  3.28  1.98  0.98  1.97
2^-6  3.13  1.99  0.99  1.99
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  2.86  1.87  0.91  1.77
2^-3  3.06  1.92  0.85  1.87
2^-4  3.37  1.94  0.93  1.91
2^-5  3.28  1.98  0.98  1.97
2^-6  3.13  1.99  0.99  1.99


#### With penalty (Problem2)

In [9]:
sigma = 5
element_type = 'P2'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))

uh0_penalty = uh0

epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.81  1.59  1.00  0.95
2^-3  1.57  1.67  0.91  0.87
2^-4  1.80  1.82  0.97  0.95
2^-5  1.91  1.91  0.99  0.98
2^-6  1.96  1.96  1.00  0.99
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  2.98  1.84  0.94  1.69
2^-3  2.59  1.99  0.93  1.81
2^-4  1.06  2.05  1.10  1.72
2^-5  0.91  1.73  1.06  1.30
2^-6  1.46  1.59  0.90  1.00
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  3.01  1.83  0.93  1.70
2^-3  3.07  1.91  0.87  1.86
2^-4  3.36  1.94  0.94  1.92
2^-5  3.25  1.98  0.99  1.97
2^-6  3.09  2.00  1.00  1.99
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  3.01  1.83  0.93  1.70
2^-3  3.07  1.91  0.87  1.86
2^-4  3.36  1.94  0.94  1.92
2^-5  3.26  1.98  0.99  1.97
2^-6  3.10  2.00  1.00  1.99
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  3.01  1.83  0.93  1.70
2^-3  3.07  1.91  0.87  1.86
2^-4  3.36  1.94  0.94  1.92
2^-5  3.26  1.98  0.99  1.97
2^-6  3.10  2.00  1.00  1.99


## Example 2

$$
u=g(x)p(y)
$$
where
$$
g(x)=\frac{1}{2}\left[\sin (\pi x)+\frac{\pi \varepsilon}{1-e^{-1 / \varepsilon}}\left(e^{-x / \varepsilon}+e^{(x-1) / \varepsilon}-1-e^{-1 / \varepsilon}\right)\right]
$$
$$
p(y)=2 y\left(1-y^{2}\right)+\varepsilon\left[l d(1-2 y)-3 \frac{q}{l}+\left(\frac{3}{l}-d\right) e^{-y / \varepsilon}+\left(\frac{3}{l}+d\right) e^{(y-1) / \varepsilon}\right]
$$
$$
l=1-e^{-1 / \varepsilon}, q=2-l \text { and } d=1 /(q-2 \varepsilon l)
$$

> epsilon needs to be set smaller in P2-penalty case

In [10]:
@LinearForm
def f_load(v, w):
    '''
    for $(f, x_{h})$
    '''
    x = w.x[0]
    y = w.x[1]
    return (
        (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
            (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) *
        (12 * y + ep *
         ((exp(-y / ep) *
           (3 / (exp(-1 / ep) - 1) + 1 /
            (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1))) / ep**2 + (exp(
                (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                 (exp(-1 / ep) + 2 * ep *
                                  (exp(-1 / ep) - 1) + 1))) / ep**2)) -
        ((pi**2 * sin(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep**2 + exp(
            (x - 1) / ep) / ep**2)) / (2 * (exp(-1 / ep) - 1))) *
        (ep * (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                    (exp(-1 / ep) + 2 * ep *
                                     (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
               (3 / (exp(-1 / ep) - 1) + 1 /
                (exp(-1 / ep) + 2 * ep *
                 (exp(-1 / ep) - 1) + 1)) - (3 * exp(-1 / ep) + 3) /
               (exp(-1 / ep) - 1) - ((2 * y - 1) * (exp(-1 / ep) - 1)) /
               (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) + 2 * y *
         (y**2 - 1)) - ep**2 *
        (((pi**4 * sin(pi * x)) / 2 - (ep * pi * (exp(-x / ep) / ep**4 + exp(
            (x - 1) / ep) / ep**4)) / (2 * (exp(-1 / ep) - 1))) *
         (ep * (exp((y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                     (exp(-1 / ep) + 2 * ep *
                                      (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                (3 / (exp(-1 / ep) - 1) + 1 /
                 (exp(-1 / ep) + 2 * ep *
                  (exp(-1 / ep) - 1) + 1)) - (3 * exp(-1 / ep) + 3) /
                (exp(-1 / ep) - 1) - ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1)) + 2 * y *
          (y**2 - 1)) - 2 *
         (12 * y + ep *
          ((exp(-y / ep) *
            (3 / (exp(-1 / ep) - 1) + 1 /
             (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1))) / ep**2 + (exp(
                 (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                  (exp(-1 / ep) + 2 * ep *
                                   (exp(-1 / ep) - 1) + 1))) / ep**2)) *
         ((pi**2 * sin(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep**2 + exp(
             (x - 1) / ep) / ep**2)) / (2 * (exp(-1 / ep) - 1))) + ep *
         (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
             (x - 1) / ep) - exp(-1 / ep) - 1)) / (2 * (exp(-1 / ep) - 1))) *
         ((exp(-y / ep) *
           (3 / (exp(-1 / ep) - 1) + 1 /
            (exp(-1 / ep) + 2 * ep * (exp(-1 / ep) - 1) + 1))) / ep**4 + (exp(
                (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                 (exp(-1 / ep) + 2 * ep *
                                  (exp(-1 / ep) - 1) + 1))) / ep**4))) * v


def exact_u(x, y):
    return -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
        (x - 1) / ep) - exp(-1 / ep) - 1)) /
             (2 *
              (exp(-1 / ep) - 1))) * (ep * (exp(
                  (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                   (exp(-1 / ep) + 2 * ep *
                                    (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                            (3 / (exp(-1 / ep) - 1) + 1 /
                                             (exp(-1 / ep) + 2 * ep *
                                              (exp(-1 / ep) - 1) + 1)) -
                                            (3 * exp(-1 / ep) + 3) /
                                            (exp(-1 / ep) - 1) -
                                            ((2 * y - 1) *
                                             (exp(-1 / ep) - 1)) /
                                            (exp(-1 / ep) + 2 * ep *
                                             (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                      (y**2 - 1))


def dexact_u(x, y):
    dux = -((pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
        (x - 1) / ep) / ep)) /
            (2 *
             (exp(-1 / ep) - 1))) * (ep * (exp(
                 (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                  (exp(-1 / ep) + 2 * ep *
                                   (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                           (3 / (exp(-1 / ep) - 1) + 1 /
                                            (exp(-1 / ep) + 2 * ep *
                                             (exp(-1 / ep) - 1) + 1)) -
                                           (3 * exp(-1 / ep) + 3) /
                                           (exp(-1 / ep) - 1) -
                                           ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                           (exp(-1 / ep) + 2 * ep *
                                            (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                     (y**2 - 1))
    duy = (sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
        (x - 1) / ep) - exp(-1 / ep) - 1)) /
           (2 * (exp(-1 / ep) - 1))) * (ep * (
               (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                           (exp(-1 / ep) - 1) + 1) +
               (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                                (exp(-1 / ep) + 2 * ep *
                                 (exp(-1 / ep) - 1) + 1))) / ep -
               (exp((y - 1) / ep) *
                (3 / (exp(-1 / ep) - 1) - 1 /
                 (exp(-1 / ep) + 2 * ep *
                  (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
    return dux, duy


def ddexact(x, y):
    duxx = ((pi**2 * sin(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep**2 + exp(
        (x - 1) / ep) / ep**2)) /
            (2 *
             (exp(-1 / ep) - 1))) * (ep * (exp(
                 (y - 1) / ep) * (3 / (exp(-1 / ep) - 1) - 1 /
                                  (exp(-1 / ep) + 2 * ep *
                                   (exp(-1 / ep) - 1) + 1)) + exp(-y / ep) *
                                           (3 / (exp(-1 / ep) - 1) + 1 /
                                            (exp(-1 / ep) + 2 * ep *
                                             (exp(-1 / ep) - 1) + 1)) -
                                           (3 * exp(-1 / ep) + 3) /
                                           (exp(-1 / ep) - 1) -
                                           ((2 * y - 1) * (exp(-1 / ep) - 1)) /
                                           (exp(-1 / ep) + 2 * ep *
                                            (exp(-1 / ep) - 1) + 1)) + 2 * y *
                                     (y**2 - 1))
    duxy = ((pi * cos(pi * x)) / 2 + (ep * pi * (exp(-x / ep) / ep - exp(
        (x - 1) / ep) / ep)) / (2 * (exp(-1 / ep) - 1))) * (ep * (
            (2 * (exp(-1 / ep) - 1)) / (exp(-1 / ep) + 2 * ep *
                                        (exp(-1 / ep) - 1) + 1) +
            (exp(-y / ep) * (3 / (exp(-1 / ep) - 1) + 1 /
                             (exp(-1 / ep) + 2 * ep *
                              (exp(-1 / ep) - 1) + 1))) / ep -
            (exp((y - 1) / ep) *
             (3 / (exp(-1 / ep) - 1) - 1 /
              (exp(-1 / ep) + 2 * ep *
               (exp(-1 / ep) - 1) + 1))) / ep) - 6 * y**2 + 2)
    duyx = duxy
    duyy = -(sin(pi * x) / 2 - (ep * pi * (exp(-x / ep) + exp(
        (x - 1) / ep) - exp(-1 / ep) - 1)) /
             (2 *
              (exp(-1 / ep) - 1))) * (12 * y + ep *
                                      ((exp(-y / ep) *
                                        (3 / (exp(-1 / ep) - 1) + 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1))) / ep**2 +
                                       (exp((y - 1) / ep) *
                                        (3 / (exp(-1 / ep) - 1) - 1 /
                                         (exp(-1 / ep) + 2 * ep *
                                          (exp(-1 / ep) - 1) + 1))) / ep**2))
    return duxx, duxy, duyx, duyy

### P1 element

#### Without penalty (Problem1)

In [11]:
refine_time = 6
epsilon_range = 5
element_type = 'P1'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))


element_type: P1
epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  0.00  0.00  0.00  0.00
2^-3  -0.00  -0.00  -0.00  -0.00
2^-4  -0.00  -0.00  -0.00  -0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.64  0.75  -0.23  0.64
2^-3  1.46  0.61  -0.44  0.52
2^-4  1.04  0.52  -0.42  0.38
2^-5  -0.13  0.39  -0.30  0.19
2^-6  -0.42  0.14  -0.13  0.03
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  1.66  0.75  -0.23  0.65
2^-3  1.53  0.61  -0.47  0.56
2^-4  1.50  0.54  -0.49  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.66  0.75  -0.23  0.65
2^-3  1.53  0.61  -0.47  0.56
2^-4  1.50  0.54  -0.49  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  1.66  0.75  -0.23  0.65
2^-3  1.53  0.61  -0.47  0.56
2^-4  1.50  0.54  -0.49  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50

#### With penalty (Problem2)

In [12]:
sigma = 5
element_type = 'P1'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))


epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.66  1.21  0.75  0.72
2^-3  1.46  1.61  0.93  0.90
2^-4  1.67  1.78  0.95  0.93
2^-5  1.84  1.89  0.97  0.96
2^-6  1.93  1.95  0.99  0.98
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  2.37  1.52  -0.59  1.36
2^-3  1.41  1.19  -0.70  0.61
2^-4  0.04  0.28  -0.13  0.01
2^-5  0.57  0.74  0.39  0.48
2^-6  1.17  1.13  0.71  0.77
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  2.16  1.40  0.49  1.31
2^-3  2.38  1.59  0.55  1.53
2^-4  2.39  1.59  0.55  1.56
2^-5  2.64  1.57  0.53  1.55
2^-6  0.73  1.50  0.52  1.51
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  2.16  1.40  0.49  1.31
2^-3  2.36  1.59  0.55  1.53
2^-4  2.28  1.59  0.55  1.56
2^-5  2.18  1.56  0.53  1.55
2^-6  2.12  1.54  0.52  1.53
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  2.16  1.40  0.49  1.31
2^-3  2.36  1.59  0.55  1.53
2^-4  2.28  1.59  0.55  1.56
2^-5  2.17  1.56  0.53  1.55
2^-6  2.09  1.54  0.52  1.53


### P2 element

#### Without penalty (Problem1)

In [13]:
refine_time = 6
epsilon_range = 5
element_type = 'P2'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))


element_type: P2
epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  -0.01  -0.00  -0.00  -0.00
2^-3  -0.01  -0.00  -0.00  -0.00
2^-4  -0.00  -0.00  -0.00  -0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.53  0.61  -0.29  0.50
2^-3  1.51  0.55  -0.40  0.46
2^-4  1.13  0.51  -0.40  0.37
2^-5  -0.23  0.38  -0.29  0.19
2^-6  -0.47  0.13  -0.12  0.03
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  1.52  0.61  -0.30  0.51
2^-3  1.53  0.56  -0.42  0.50
2^-4  1.51  0.53  -0.47  0.50
2^-5  1.50  0.51  -0.49  0.50
2^-6  1.50  0.51  -0.49  0.50
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.52  0.61  -0.30  0.51
2^-3  1.53  0.56  -0.42  0.50
2^-4  1.51  0.53  -0.47  0.50
2^-5  1.50  0.51  -0.49  0.50
2^-6  1.50  0.51  -0.49  0.50
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  1.52  0.61  -0.30  0.51
2^-3  1.53  0.56  -0.42  0.50
2^-4  1.51  0.53  -0.47  0.50
2^-5  1.50  0.51  -0.49  0.50
2^-6  1.50  0.51  -0.49  

#### With penalty (Problem2)

In [14]:
sigma = 5
element_type = 'P2'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))

epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  1.44  1.35  0.87  0.83
2^-3  1.59  1.57  0.90  0.86
2^-4  1.78  1.73  0.94  0.91
2^-5  1.90  1.87  0.97  0.95
2^-6  1.95  1.94  0.99  0.98
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.39  1.26  -1.04  0.86
2^-3  0.37  -0.15  -0.73  -0.48
2^-4  0.32  0.21  -0.12  -0.01
2^-5  0.67  0.80  0.40  0.50
2^-6  1.20  1.15  0.72  0.78
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  3.10  1.91  0.87  1.81
2^-3  3.05  1.95  0.96  1.91
2^-4  1.43  1.96  0.99  1.97
2^-5  0.14  1.85  1.00  1.96
2^-6  0.02  1.29  0.78  1.54
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  3.14  1.91  0.87  1.81
2^-3  3.35  1.96  0.96  1.91
2^-4  3.21  1.99  0.99  1.97
2^-5  2.96  2.00  1.00  1.99
2^-6  1.58  2.00  1.00  2.00
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  3.14  1.91  0.87  1.81
2^-3  3.35  1.96  0.96  1.91
2^-4  3.23  1.99  0.99  1.97
2^-5  3.09  2.00  1.00  1.99
2^-6  3.02  2.00  1.00  2.00


## Example3

$$
u^{0}\left(x_{1}, x_{2}\right)=\sin \left(\pi x_{1}\right) \sin \left(\pi x_{2}\right)
$$
$$
f\left(x_{1}, x_{2}\right)=-\Delta u^{0}=2 \pi^{2} \sin \left(\pi x_{1}\right) \sin \left(\pi x_{2}\right)
$$

> epsilon needs to be set smaller in P2-penalty case

In [22]:
def exact_u(x, y):
    return sin(pi * x) * sin(pi * y)


def dexact_u(x, y):
    dux = pi * cos(pi * x) * sin(pi * y)
    duy = pi * cos(pi * y) * sin(pi * x)
    return dux, duy


def ddexact(x, y):
    duxx = -pi**2 * sin(pi * x) * sin(pi * y)
    duxy = pi * cos(pi * x) * pi * cos(pi * y)
    duyx = duxy
    duyy = -pi**2 * sin(pi * y) * sin(pi * x)
    return duxx, duxy, duyx, duyy

In [23]:
@Functional
def L2uError(w):
    x, y = w.x
    return (w.w - exact_u(x, y))**2


def get_DuError(basis, u):
    duh = basis.interpolate(u).grad
    x = basis.global_coordinates().value
    dx = basis.dx  # quadrature weights
    dux, duy = dexact_u(x[0], x[1])
    return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))


def get_D2uError(basis, u):
    dduh = basis.interpolate(u).hess
    x = basis.global_coordinates().value
    dx = basis.dx
    duxx, duxy, duyx, duyy = ddexact(x[0], x[1])
    return np.sqrt(
        np.sum(((dduh[0][0] - duxx)**2 + (dduh[0][1] - duxy)**2 +
                (dduh[1][1] - duyy)**2 + (dduh[1][0] - duyx)**2) * dx))


@LinearForm
def f_load(v, w):
    pix = pi * w.x[0]
    piy = pi * w.x[1]
    return (2 * pi**2 * sin(pix) * sin(piy)) * v


@BilinearForm
def laplace(u, v, w):
    '''
    for $(\nabla w_{h}, \nabla \chi_{h})$
    '''
    return dot(grad(u), grad(v))

### P1 element

#### Without penalty (Problem1)

In [24]:
refine_time = 6
epsilon_range = 5
element_type = 'P1'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))


element_type: P1
epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  0.00  0.00  0.01  0.01
2^-3  -0.00  -0.00  0.00  0.00
2^-4  -0.00  -0.00  -0.00  0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.72  0.81  -0.19  0.70
2^-3  1.49  0.65  -0.44  0.56
2^-4  1.02  0.53  -0.42  0.39
2^-5  -0.14  0.39  -0.30  0.19
2^-6  -0.41  0.14  -0.13  0.03
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  1.74  0.81  -0.20  0.71
2^-3  1.57  0.65  -0.46  0.60
2^-4  1.51  0.55  -0.49  0.52
2^-5  1.51  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.74  0.81  -0.20  0.71
2^-3  1.57  0.65  -0.46  0.60
2^-4  1.51  0.55  -0.49  0.52
2^-5  1.51  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  1.74  0.81  -0.20  0.71
2^-3  1.57  0.65  -0.46  0.60
2^-4  1.51  0.55  -0.49  0.52
2^-5  1.51  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.50  0.50


#### With penalty (Problem2)

In [25]:
sigma = 5
element_type = 'P1'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))


epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  0.00  0.00  0.00  0.00
2^-3  -0.00  0.00  0.00  0.00
2^-4  -0.00  -0.00  0.00  0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  2.15  1.44  0.54  1.34
2^-3  1.90  1.48  0.58  1.42
2^-4  0.34  0.18  -0.65  0.10
2^-5  -0.48  -0.54  -1.30  -0.71
2^-6  -0.42  -0.25  -0.63  -0.39
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  2.19  1.46  0.53  1.37
2^-3  2.39  1.67  0.63  1.62
2^-4  2.26  1.67  0.62  1.65
2^-5  2.13  1.63  0.58  1.61
2^-6  2.03  1.58  0.55  1.57
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  2.19  1.46  0.53  1.37
2^-3  2.39  1.67  0.63  1.62
2^-4  2.26  1.67  0.62  1.65
2^-5  2.13  1.63  0.58  1.61
2^-6  2.06  1.58  0.55  1.57
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  2.19  1.46  0.53  1.37
2^-3  2.39  1.67  0.63  1.62
2^-4  2.26  1.67  0.62  1.65
2^-5  2.13  1.63  0.58  1.61
2^-6  2.06  1.58  0.55  1.57


### P2 element

#### Without penalty (Problem1)

In [19]:
refine_time = 6
epsilon_range = 5
element_type = 'P2'
print('element_type:', element_type)
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j*2)

    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time+1):
        
        m.refine()
        uh0, basis = solve_problem1(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)
        
    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us
    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
#         print(
#             '2^-' + str(i + 2), ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(
#                 L2s[i + 1], H1s[i + 1],
#                 H2s[i + 1],
#                 epus[i + 1]))


element_type: P2
epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  -0.01  -0.00  0.00  0.00
2^-3  -0.01  -0.00  -0.00  -0.00
2^-4  -0.00  -0.00  -0.00  -0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  1.59  0.67  -0.28  0.56
2^-3  1.54  0.58  -0.39  0.48
2^-4  1.12  0.51  -0.39  0.38
2^-5  -0.25  0.38  -0.29  0.19
2^-6  -0.47  0.13  -0.12  0.03
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  1.59  0.67  -0.28  0.57
2^-3  1.56  0.58  -0.41  0.52
2^-4  1.52  0.53  -0.47  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.49  0.50
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  1.59  0.67  -0.28  0.57
2^-3  1.56  0.58  -0.41  0.52
2^-4  1.52  0.53  -0.47  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.49  0.50
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  1.59  0.67  -0.28  0.57
2^-3  1.56  0.58  -0.41  0.52
2^-4  1.52  0.53  -0.47  0.51
2^-5  1.50  0.52  -0.49  0.50
2^-6  1.50  0.51  -0.49  0.

#### With penalty (Problem2)

In [20]:
sigma = 5
element_type = 'P2'
for j in range(epsilon_range):
    epsilon = 1 * 10**(-j * 2)
    ep = epsilon
    L2_list = []
    Du_list = []
    D2u_list = []
    h_list = []
    epu_list = []
    m = MeshTri()

    for i in range(1, refine_time + 1):

        m.refine()
        uh0, basis = solve_problem2(m, element_type)
        U = basis['u'].interpolate(uh0).value

        L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
        Du = get_DuError(basis['u'], uh0)
        H1u = Du + L2u
        D2u = get_D2uError(basis['u'], uh0)
        H2u = Du + L2u + D2u
        epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
        h_list.append(m.param())
        Du_list.append(Du)
        L2_list.append(L2u)
        D2u_list.append(D2u)
        epu_list.append(epu)

    hs = np.array(h_list)
    L2s = np.array(L2_list)
    Dus = np.array(Du_list)
    D2us = np.array(D2u_list)
    epus = np.array(epu_list)
    H1s = L2s + Dus
    H2s = H1s + D2us

    print('epsilon =', epsilon)
    print('  h    L2u   H1u   H2u   epu')
    for i in range(H2s.shape[0] - 1):
        print(
            '2^-' + str(i + 2), ' {:.2f}  {:.2f}  {:.2f}  {:.2f}'.format(
                -np.log2(L2s[i + 1] / L2s[i]), -np.log2(H1s[i + 1] / H1s[i]),
                -np.log2(H2s[i + 1] / H2s[i]),
                -np.log2(epus[i + 1] / epus[i])))
        
#         print(
#             '2^-' + str(i + 2),
#             ' {:.5f}  {:.5f}  {:.5f}  {:.5f}'.format(L2s[i + 1], H1s[i + 1],
#                                                      H2s[i + 1], epus[i + 1]))

epsilon = 1
  h    L2u   H1u   H2u   epu
2^-2  -0.01  -0.01  -0.00  -0.00
2^-3  -0.00  -0.00  -0.00  -0.00
2^-4  -0.00  -0.00  -0.00  -0.00
2^-5  -0.00  -0.00  -0.00  -0.00
2^-6  -0.00  -0.00  -0.00  -0.00
epsilon = 0.01
  h    L2u   H1u   H2u   epu
2^-2  3.33  1.94  0.91  1.78
2^-3  0.44  0.66  0.59  0.66
2^-4  -0.90  -1.04  -1.67  -1.11
2^-5  -0.77  -0.65  -1.38  -0.81
2^-6  -0.48  -0.26  -0.63  -0.39
epsilon = 0.0001
  h    L2u   H1u   H2u   epu
2^-2  3.15  1.91  0.86  1.79
2^-3  3.40  1.95  0.95  1.90
2^-4  3.30  1.98  0.99  1.97
2^-5  3.13  2.00  1.00  1.99
2^-6  1.27  1.94  1.00  1.94
epsilon = 1e-06
  h    L2u   H1u   H2u   epu
2^-2  3.15  1.91  0.86  1.79
2^-3  3.40  1.95  0.95  1.90
2^-4  3.30  1.98  0.99  1.97
2^-5  3.12  2.00  1.00  1.99
2^-6  3.04  2.00  1.00  2.00
epsilon = 1e-08
  h    L2u   H1u   H2u   epu
2^-2  3.15  1.91  0.86  1.79
2^-3  3.40  1.95  0.95  1.90
2^-4  3.30  1.98  0.99  1.97
2^-5  3.12  2.00  1.00  1.99
2^-6  3.04  2.00  1.00  2.00
